# GEMM on GPU

## 1. Set-up

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-kpan02' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-kpan02
M	src/ops.py
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb   4-gemm_gpu.ipynb	    README.md
2-conv1d_gpu.ipynb   5-conv2d_dw_gpu.ipynb  src
3-conv1d_fpga.ipynb  leaderboard_id.txt     tests


## 2. Install TVM

In [6]:
!pip install numpy==1.24.3
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels


## 3. Check the implementation of `make_gemm_gpu_scheduler` function in `src.ops`

The function implements General Matrix Multiply (GEMM) on GPU. You should use TVM to optimize it.

Let $A \in \mathbb{R}^{m \times k}$, $W \in \mathbb{R}^{k \times n}$, and $B \in \mathbb{R}^{m \times n}$, then
$$
B = A \times W
$$
Please see the numpy matmul function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).

The `make_gemm_gpu_scheduler` takes $m$, $k$, and $n$. The first matrix is $m \times k$, the second matrix is $k \times n$, and the output matrix is $m \times n$.

The function returns both the TVM scheduler and the TVM opterator for
1. Input $a$
2. Input $w$
3. Output $b$

The scheduler should be able to used to build a function with signature $func(a, w, b)$.
Please see the following cells for usage.

In [7]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_gemm_gpu_scheduler

M = 1024
N = 512
K = 2048
dtype = 'float32'
a_np = np.random.rand(M, K).astype(dtype)
w_np = np.random.rand(K, N).astype(dtype)
b_np = np.matmul(a_np, w_np)

s, A, W, B = make_gemm_gpu_scheduler(M, K, N)
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M, N), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"GEMM TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [[511.21603 523.69086 533.4453  ... 505.56244 516.326   522.1519 ]
 [504.8399  523.59705 527.8334  ... 500.84293 512.91907 509.86575]
 [499.38998 512.8939  512.59033 ... 486.7307  491.46466 508.764  ]
 ...
 [514.9759  528.45703 534.9043  ... 508.16986 520.47095 525.9487 ]
 [505.64557 527.23987 523.86597 ... 495.59018 509.95844 515.9603 ]
 [508.211   522.7613  529.2311  ... 504.02762 514.0861  518.62836]]
Output: [[511.21637 523.6911  533.4453  ... 505.5621  516.32605 522.152  ]
 [504.84006 523.59717 527.834   ... 500.84293 512.9185  509.86615]
 [499.38956 512.89355 512.5899  ... 486.7312  491.46457 508.76407]
 ...
 [514.9762  528.45715 534.9042  ... 508.1693  520.4713  525.949  ]
 [505.6456  527.2397  523.86584 ... 495.5904  509.95786 515.96045]
 [508.2114  522.76166 529.23145 ... 504.02737 514.0859  518.62805]]
GEMM TVM: 24.762304 ms


In [8]:
import timeit
import numpy as np
numpy_timer = timeit.Timer(lambda: np.matmul(a_np, w_np))
numpy_time = numpy_timer.timeit(number=1) * 1000
print(f"Numpy Reference Runtime: {numpy_time:.6f} ms")

Numpy Reference Runtime: 52.949851 ms


In [9]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 2048), "float32"), B: T.Buffer((2048, 512), "float32"), C: T.Buffer((1024, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_x = T.launch_thread("blockIdx.x", 64)
        A_shared = T.allocate([128], "float32", "shared")
        B_shared = T.allocate([128], "float32", "shared")
        blockIdx_y = T.launch_thread("blockIdx.y", 32)
        threadIdx_x = T.env_thread("threadIdx.x")
        threadIdx_y = T.env_thread("threadIdx.y")
        C_1 = T.Buffer((524288,), data=C.data)
        with T.launch_thread(threadIdx_x, 16):
            T.launch_thread(threadIdx_y, 16)
            C_1[blockIdx_x * 8192 + threadIdx_x * 512 + blockIdx_y * 16 + threadIdx_y] = T.float32(0)
        for k_outer in range(256):
            threadIdx_x_1 = T.env_thread("threadIdx.x")
            

In [10]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_gemm_gpu.py

/content/gdrive/MyDrive/ece5545/a3-kpan02
============================= test session starts ==============================
platform linux -- Python 3.11.12, pytest-8.3.5, pluggy-1.5.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-kpan02
plugins: typeguard-4.4.2, anyio-4.9.0, langsmith-0.3.24
collected 20 items                                                             

tests/test_gemm_gpu.py ....................                              [100%]

============================= 20 passed in 23.38s ==============================
